### Load SP_diagnosis from BQ

In [1]:
# example: read from BigQuery chart to numpy
from google.cloud import bigquery
from google.cloud.bigquery import dbapi;
import numpy as np
import pandas as pd



client = bigquery.Client("som-nero-phi-jonc101"); # Project identifier
conn = dbapi.connect(client);
cursor = conn.cursor();
query = "select * from `som-nero-phi-jonc101.Clinical_Collaborative_Filtering.spec_dx`"; # Example dataset table


# to DataFrame

df_all = (
    client.query(query)
    .result()
    .to_dataframe()
)



/Users/morteza/anaconda3/lib/python3.6/site-packages/google/auth/_default.py:69: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Remove unnecessary columns and rows

In [2]:
print(df_all.head())

# only take PC_enc and proc_id
df = df_all[['PC_enc', 'SP_diagnosis']]

# remove duplicate rows:
df = df.drop_duplicates()

print(df.head())

     jc_uid        PC_enc     PC_app_datetime     PC_ref_datetime  \
0  JCcb6607  131239705939 2017-11-28 09:15:00 2017-11-28 10:21:00   
1  JCcb6607  131239705939 2017-11-28 09:15:00 2017-11-28 10:21:00   
2  JCcb6607  131239705939 2017-11-28 09:15:00 2017-11-28 10:21:00   
3  JCcb6607  131239705939 2017-11-28 09:15:00 2017-11-28 10:21:00   
4  JCcb6607  131239705939 2017-11-28 09:15:00 2017-11-28 10:21:00   

         SP_enc     SP_app_datetime                        SP_diagnosis  
0  131240863187 2018-03-17 09:00:00  Nonscarring hair loss, unspecified  
1  131240863187 2018-03-17 09:00:00                   Acne, unspecified  
2  131240863187 2018-03-17 09:00:00                            Alopecia  
3  131240863187 2018-03-17 09:00:00                          Overweight  
4  131240863187 2018-03-17 09:00:00                          Overweight  
         PC_enc                        SP_diagnosis
0  131239705939  Nonscarring hair loss, unspecified
1  131239705939                   Acn

### Create the table of unique encounter id

In [4]:
unq_PC_enc = df.groupby(['PC_enc']).apply(len).reset_index().rename(columns={'SP_diagnosis':'SP_diagnosis', 0:'num_unq_proc'}).sort_values('num_unq_proc',ascending=False)  
print(len(unq_PC_enc))
print(unq_PC_enc.head())
print('All of the encounters with at least one diagnosis: ',len(unq_PC_enc))
print('Average diagnosis per encounter: ', np.mean(unq_PC_enc.num_unq_proc.to_numpy()))
print('Min # diagnosis per encounter: ', np.min(unq_PC_enc.num_unq_proc.to_numpy()))
print('Max # diagnosis per encounter: ', np.max(unq_PC_enc.num_unq_proc.to_numpy()))

12133
             PC_enc  num_unq_proc
6150   131167856697            41
8273   131229230099            28
4597   131048548315            28
5476   131091604362            25
10017  131243949605            24
All of the encounters with at least one diagnosis:  12133
Average diagnosis per encounter:  3.8549410698096103
Min # diagnosis per encounter:  1
Max # diagnosis per encounter:  41


### Sort according to the most frequent labs (not necessarily how important they are)

In [6]:


Top_proc = ['iabetes' # diabetes mellitus, Type II or unspecified type diabetes mellitus, Prediabetes
            ,'nodule' # Thyroid nodule
            ,'thyroidism' #hypothyroidism (Hypothyroidism, Hyperparathyroidism)
            ,'goiter' #Nontoxic uninodular goiter,  Nontoxic multinodular goiter
            ,'ypertension' #essential hypertension (hypertension)
            , 'itamin'# Vitamin D deficiency (Vitamin D insufficiency)
            , 'steop' #Osteoporosis (Osteopenia)
            , 'gland'#Malignant neoplasm of thyroid gland (CMS-HCC) (Malignant neoplasm of thyroid gland)
            , 'hyrotoxicosis'# Thyrotoxicosis
            , 'Thyroid cancer'# Thyroid cancer (CMS-HCC)
            , 'glucose'# abnormal glucose
            , 'ypercalcemia' # Hypercalcemia
            , 'yslipidemia' #Dyslipidemia
            , 'carcinoma' #Papillary thyroid carcinoma (CMS-HCC)
            , 'Hashimoto' #Hashimoto's thyroiditis
            , 'ypoglycemia'# Hypoglycemia
            , 'irsutism'# Hirsutism
            , 'PCOS' # PCOS (polycystic ovarian syndrome)
            , 'Graves'#Graves disease
            , 'fatigue'#Chronic fatigue syndrome
            , 'ymphocytic'# Chronic lymphocytic thyroiditis
            , 'cartilage' #Disorder of bone and cartilage
           ]

N_diagnosis = len(Top_proc)
print(N_diagnosis)
#print(Top_proc)

#df=df.sort_values('num',ascending=False)

22


### Create Feature Matrix of Lab Results

In [14]:
def my_func(df):
        return int(len(df.query('SP_diagnosis.str.contains("iabetes")'))>0)
    
F_df = df.groupby(['PC_enc']).apply(my_func).reset_index().rename(columns={'SP_diagnosis':'SP_diagnosis', 0:'F'+str(i)}) 


TypeError: 'Series' objects are mutable, thus they cannot be hashed

In [13]:
print(df.columns)
import sys

for i in range(N_diagnosis):
    
    # define the function: applies on the lab results of each encounter
    def my_func(df):
        return int(len(df.query('SP_diagnosis.str.contains("iabetes")'))>0)
    
    F_df = df.groupby(['PC_enc']).apply(my_func).reset_index().rename(columns={'PC_diagnosis':'PC_diagnosis', 0:'F'+str(i)}) 
    unq_PC_enc = pd.merge(unq_PC_enc, F_df, how='left', on='PC_enc')
    
    # report i 
    sys.stdout.write('\r')
    sys.stdout.write(str(i/N_proc*100)+'%')
    sys.stdout.flush()


print(unq_PC_enc.head())


Index(['PC_enc', 'SP_diagnosis'], dtype='object')


TypeError: 'Series' objects are mutable, thus they cannot be hashed

### Save the Results into CSV

In [44]:
print(len(unq_PC_enc))
#print(unq_PC_enc.sort_values('F0',ascending=False))
#print(np.sum(unq_PC_enc['F0'].to_numpy()))
unq_PC_enc.to_csv('PC_lab_results.csv', index=False)

16027


### Test my_func

In [90]:
def my_func(dfi):
    a = int(len(dfi[dfi['proc_id']==2220])>0)
    return a #pd.DataFrame({'main':dfi, 'func': a })

#print(df.columns)
#print(df.head())
F_df  = df.groupby('PC_enc').apply(my_func).reset_index().rename(columns={'PC_enc':'PC_enc', 0:'F'}) 
print('Check')
df_temp = pd.merge(unq_PC_enc, F_df, how='left', on='PC_enc')
print(np.sum(df_temp['F'].to_numpy()))
print(df_temp.head())

Check
4913
         PC_enc  num_unq_proc  F
0  131094507503            32  1
1  131049295970            30  1
2  131182751483            29  1
3  131235992610            28  1
4  131194373585            27  1
